In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

In [2]:
class DataLoader:
    req_columns = ['CH_TIMESTAMP', 'CH_SYMBOL', 'CH_TRADE_HIGH_PRICE', 'CH_TRADE_LOW_PRICE', 'CH_OPENING_PRICE', 'CH_CLOSING_PRICE', 'CH_LAST_TRADED_PRICE', 'CH_PREVIOUS_CLS_PRICE', 'CH_TOT_TRADED_QTY', 'CH_52WEEK_HIGH_PRICE', 'CH_52WEEK_LOW_PRICE']
    new_column_names = ['date', 'symbol', 'high', 'low', 'open', 'close', 'ltp', 'prev_close', 'volume', 'high_52w', 'low_52w']

    @staticmethod
    def load_data(script_name, start_date, end_date, series="EQ"):
        df = equity_history(script_name, series, start_date, end_date)[DataLoader.req_columns]
        df.columns = DataLoader.new_column_names
        return df

In [10]:
def load_volume_sma(df):
    df['vol_sma_10d'] = df.volume.rolling(window=10).mean()
    return df

def load_emas(df):
    ema_spans = [9, 21, 30, 50, 100, 12, 26]
    for i in ema_spans:
        df[f'ema_{i}d'] = df.close.ewm(span=i).mean()
    return df

def load_macd(df):
    df['macd'] = df.ema_12d - df.ema_26d
    df['macd_9d_signal'] = df.macd.ewm(9).mean()
    return df
    
def load_bollinger_bands(df):
    df['sma_20d'] = df.close.rolling(window=20).mean()
    df['bb_upper'] = df.sma_20d + 2 * df.close.rolling(20).std()
    df['bb_lower'] = df.sma_20d - 2 * df.close.rolling(20).std()
    return df

def load_rsi(df):
    df['gain_pts'] = np.where(df["close"].diff() > 0, df["close"].diff(), 0)
    df['loss_pts'] = np.where(df["close"].diff() < 0, df["close"].diff().abs(), 0)
    df['gain_avg'] = df['gain_pts'].ewm(alpha=1/14, adjust=True, min_periods=14).mean()
    df['loss_avg'] = df['loss_pts'].ewm(alpha=1/14, adjust=True, min_periods=14).mean()
    df['rsi'] = 100 - (100 / (1 + df['gain_avg'] / df['loss_avg']))
    return df

def load_ATR(df):
    df['true_range'] = np.maximum.reduce([
        df['high'] - df['low'], 
        (df['high'] - df['prev_close']).abs(), 
        (df['low'] - df['prev_close']).abs()
    ])
    df['ATR'] = df['true_range'].ewm(alpha=1/14).mean()
    return df

def load_ADX(df):
    df["plus_DM"] = df['high'].diff(1)
    df["minus_DM"] = -df['low'].diff(1)
    df['plus_DX'] = np.where(
        np.logical_and(
            df['plus_DM'] > df["minus_DM"],
            df['plus_DM'] > 0
        ), df["plus_DM"], 0)
    df['minus_DX'] = np.where(
        np.logical_and(
            df['minus_DM'] > df["plus_DM"],
            df['minus_DM'] > 0
        ), df["minus_DM"], 0)
    
    df['smooth_DXp'] = df["plus_DX"].ewm(alpha=1/14).mean()
    df['smooth_DXm'] = df["minus_DX"].ewm(alpha=1/14).mean()
    
    df['plus_DI'] = df["smooth_DXp"] * 100 / df["ATR"]
    df['minus_DI'] = df["smooth_DXm"] * 100 / df['ATR']
    
    df['DX'] = ((df['plus_DI'] - df['minus_DI']).abs() * 100 / (df['plus_DI'] + df['minus_DI'])).abs()
    df['ADX'] = df['DX'].ewm(alpha=1/14).mean()
    
    return df

def load_aroon_self(df):
    """group by and joins"""
    df["low_25d"] = df["low"].rolling(window=25, min_periods=25).min()
    df["high_25d"] = df["high"].rolling(window=25, min_periods=25).max()
    df["row_number"] = df.reset_index().index
    
    min_dates = df.reset_index().groupby("low_25d").agg({"row_number": "min"})
    max_dates = df.reset_index().groupby("high_25d").agg({"row_number": "min"})
    
    df = df.reset_index().join(min_dates, on="low_25d", rsuffix="_low_25d").join(max_dates, on="high_25d", rsuffix="_high_25d")
    
    df["days_since_25d_high"] = df["row_number"] - df["row_number_high_25d"]
    df["days_since_25d_low"] = df["row_number"] - df["row_number_low_25d"]
    
    df["aroon_up"] = (25 - df["days_since_25d_high"]) * 100 / 25
    df["aroon_down"] = (25 - df["days_since_25d_low"]) * 100 / 25
    
    return df

def load_aroon(df, period):
    """using argmax"""
    df["period"] = period
    df["days_since_period_high"] = df["high"].rolling(window=period).apply(lambda x: period - np.argmax(x) - 1)
    df["days_since_period_low"] = df["low"].rolling(window=period).apply(lambda x: period - np.argmin(x) - 1)
    
    df["aroon_up"] = (period - df["days_since_period_high"]) * 100 / period
    df["aroon_down"] = (period - df["days_since_period_low"]) * 100 / period
    
    return df

In [4]:
script_name = 'infy'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 366)

In [5]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()
df = df.set_index("date")

In [11]:
for fn in [load_volume_sma, load_emas, load_macd, load_bollinger_bands, load_rsi, load_ATR, load_ADX]:
    df = fn(df)
    
df = load_aroon(df, period=14)

In [ ]:
df